<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [12]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
AQI_API_KEY = os.getenv("AQI_API_KEY")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [13]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login(project='mlfs_lab1')
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163420
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [14]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=4,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [15]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,61.0,china,shanghai,jinganjiancezhan,2024-11-11,https://api.waqi.info/feed/@486


In [16]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [17]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

# yesterday's pm25 value
df_temp = air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(2)
df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=1)
df_temp['date'] = pd.to_datetime(df_temp['date']).dt.tz_localize(None)
# add pm25 value from yesterday
daily_df = pd.merge(daily_df, df_temp[['date','pm25']], how='left', on="date")
daily_df = daily_df.rename(columns={"pm25": "pm25_yesterday"})

# today's pm25 value
pm25_value = aq_today_df.iloc[0]['pm25']
today_date = aq_today_df.iloc[0]['date']
next_day = today_date + pd.Timedelta(days=1)
condition = (daily_df['date'] == next_day)
# add pm25 value from today
daily_df.loc[condition, 'pm25_yesterday'] = pm25_value

daily_df

Coordinates 31.25°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.34s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_yesterday
0,2024-11-11,15.100000,0.0,9.290511,35.537766,shanghai,61.0
1,2024-11-12,15.950000,0.0,9.178235,64.440094,shanghai,61.0
2,2024-11-13,16.500000,0.0,10.495713,95.906044,shanghai,NaN
3,2024-11-14,19.700001,0.0,7.636753,135.000107,shanghai,NaN
4,2024-11-15,19.549999,0.2,8.557102,75.379089,shanghai,NaN
5,2024-11-16,19.450001,0.0,7.754637,68.198532,shanghai,NaN
6,2024-11-17,15.950000,0.0,26.552288,12.528798,shanghai,NaN
7,2024-11-18,12.500000,0.0,16.781561,22.714453,shanghai,NaN
8,2024-11-19,12.700000,0.0,12.303366,20.556128,shanghai,NaN
9,2024-11-20,13.500000,0.0,8.699793,65.556061,shanghai,NaN


In [18]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
 6   pm25_yesterday               2 non-null      float32       
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 492.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df, write_options={"wait_for_job": True})

# data = air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(10)
# data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.21s) 


,date,pm25,country,city,street,url
3721,2024-11-02 00:00:00+00:00,103.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
708,2024-11-03 00:00:00+00:00,140.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
540,2024-11-04 00:00:00+00:00,124.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
348,2024-11-05 00:00:00+00:00,45.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1286,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1370,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3120,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,43.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3951,2024-11-10 00:00:00+00:00,61.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3952,2024-11-11 00:00:00+00:00,61.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


In [ ]:
# Insert new data
weather_fg.insert(daily_df, write_options={"wait_for_job": True})

# data = weather_fg.read().sort_values(by=['date'], ascending=True).tail(10)
# data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.09s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_yesterday
3953,2024-11-11 00:00:00+00:00,15.100000,0.0,9.290511,35.537766,shanghai,61.0
3957,2024-11-12 00:00:00+00:00,15.950000,0.0,9.178235,64.440094,shanghai,61.0
3959,2024-11-13 00:00:00+00:00,16.500000,0.0,10.495713,95.906044,shanghai,NaN
3954,2024-11-14 00:00:00+00:00,19.700001,0.0,7.636753,135.000107,shanghai,NaN
3950,2024-11-15 00:00:00+00:00,19.549999,0.2,8.557102,75.379089,shanghai,NaN
3955,2024-11-16 00:00:00+00:00,19.450001,0.0,7.754637,68.198532,shanghai,NaN
3956,2024-11-17 00:00:00+00:00,15.950000,0.0,26.552288,12.528798,shanghai,NaN
3951,2024-11-18 00:00:00+00:00,12.500000,0.0,16.781561,22.714453,shanghai,NaN
3958,2024-11-19 00:00:00+00:00,12.700000,0.0,12.303366,20.556128,shanghai,NaN
3960,2024-11-20 00:00:00+00:00,13.500000,0.0,8.699793,65.556061,shanghai,NaN


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
